In [2]:
import bql
bq = bql.Service()

### Single Data Item

In [5]:
px_last = bq.data.px_last() 
ticker = ['IBM US Equity', 'AAPL US Equity'] 
 
request = bql.Request(ticker, px_last) #Universe first 
 
response = bq.execute(request) 
response[0].df() 

,DATE,CURRENCY,PX_LAST()
ID,,,
IBM US Equity,2020-05-01,USD,122.790001
AAPL US Equity,2020-05-01,USD,291.211395


### Multiple Data Items

In [7]:
px_last = bq.data.px_last()
px_open = bq.data.px_open() 
px_low = bq.data.px_low() 
px_high = bq.data.px_high() 

fields = [px_last, px_open, px_low, px_high] 
tickers = ['IBM US Equity', 'AAPL US Equity'] 

request = bql.Request(ticker, fields)
response = bq.execute(request)

In [8]:
response[0].df()

,DATE,CURRENCY,PX_LAST()
ID,,,
IBM US Equity,2020-05-01,USD,122.879997
AAPL US Equity,2020-05-01,USD,291.444000


In [10]:
df1 = bql.combined_df(response)
df1

,DATE,CURRENCY,PX_LAST(),PX_OPEN(),PX_LOW(),PX_HIGH()
ID,,,,,,
IBM US Equity,2020-05-01,USD,122.879997,123.190002,122.230003,123.190002
AAPL US Equity,2020-05-01,USD,291.444000,286.250000,285.850006,293.570007


### Time Series

In [13]:
import datetime

start_dt = datetime.datetime(2019, 1, 1) 
end_dt = datetime.datetime(2019, 12, 31)
last = bq.data.px_last(start=start_dt, end=end_dt)

tickers = ['IBM US Equity', 'AAPL US Equity']
request = bql.Request(tickers, last)
response = bq.execute(request)

In [15]:
#df1 = bql.combined_df(response)
#df1
temp = response[0].df()
temp.head(10)

,DATE,CURRENCY,"PX_LAST(start=2019-01-01,end=2019-12-31)"
ID,,,
IBM US Equity,2019-01-01,USD,NaN
IBM US Equity,2019-01-02,USD,115.21
IBM US Equity,2019-01-03,USD,112.91
IBM US Equity,2019-01-04,USD,117.32
IBM US Equity,2019-01-05,USD,NaN
IBM US Equity,2019-01-06,USD,NaN
IBM US Equity,2019-01-07,USD,118.15
IBM US Equity,2019-01-08,USD,119.83
IBM US Equity,2019-01-09,USD,120.69


### User Defined Field (mathematical operation)

In [16]:
buy_rec = bq.data.tot_buy_rec()
tot_rec = bq.data.tot_analyst_rec() 
buy_pct = buy_rec / tot_rec * 100 
 
tickers = ['IBM US Equity', 'VOD LN Equity', 'RBS LN Equity'] 
 
request = bql.Request(tickers, buy_pct) 
response = bq.execute(request)

In [18]:
temp = response[0].df()
temp

,DATE,(TOT_BUY_REC()/TOT_ANALYST_REC())*100
ID,,
IBM US Equity,2020-05-01,28.571429
VOD LN Equity,2020-05-01,73.076923
RBS LN Equity,2020-05-01,58.333333


### params ~ with()

In [22]:
params = {'start': '2019-01-01', 'end': '2019-12-31',  
          'frq': 'd', 'fill': 'prev'}

In [25]:
px_last = bq.data.px_last(**params) 
px_open = bq.data.px_open(**params) 
px_low = bq.data.px_low(**params) 
px_high = bq.data.px_high(**params) 
 
fields = [px_last, px_open, px_low, px_high] 
ticker = ['IBM US Equity'] 
 
request = bql.Request(ticker, fields)
response = bq.execute(request)

In [27]:
temp = response[0].df()
temp.head()

,DATE,CURRENCY,"PX_LAST(start=2019-01-01,end=2019-12-31,frq=PER.d,fill=FILL.prev)"
ID,,,
IBM US Equity,2019-01-01,USD,113.67
IBM US Equity,2019-01-02,USD,115.21
IBM US Equity,2019-01-03,USD,112.91
IBM US Equity,2019-01-04,USD,117.32
IBM US Equity,2019-01-05,USD,117.32


### Universe

In [38]:
bond_univ = bq.univ.bonds('JPM US Equity') 
 
px_last = bq.data.px_last()
 
request = bql.Request(bond_univ, px_last)
response = bq.execute(request)

In [35]:
temp = response[0].df()
#temp.describe()
temp.head()

,DATE,CURRENCY,PX_LAST()
ID,,,
BH840818 Corp,2020-05-01,None,101.290001
BH840819 Corp,2020-05-01,None,101.894997
BH840822 Corp,2020-05-01,None,103.588997
EH320991 Corp,2020-05-01,None,91.912003
ZP532072 Corp,2020-05-01,None,89.000000


In [44]:
index_members = bq.univ.members('INDU Index')

tickers = bq.data.long_comp_name()
px_last = bq.data.px_last()

fields = [tickers, px_last]
request = bql.Request(index_members, fields)
response = bq.execute(request)

In [49]:
df1 = bql.combined_df(response)
df1.sample(5)


,LONG_COMP_NAME(),DATE,CURRENCY,PX_LAST()
ID,,,,
CSCO UW Equity,Cisco Systems Inc,NaT,None,NaN
JPM UN Equity,JPMorgan Chase & Co,NaT,None,NaN
GS UN Equity,NaN,2020-05-01,USD,177.580002
UNH UN Equity,NaN,2020-05-01,USD,282.059998
DOW UN Equity,NaN,2020-05-01,USD,33.610001


In [51]:
options = bq.data.bs_options_granted()
group = bq.func.group(options) 
count = bq.func.count(group) 
 
univ = bq.univ.members('SPX Index') 
 
request = bql.Request(univ, count)
response = bq.execute(request)

In [53]:
df1 = bql.combined_df(response)
df1

,REVISION_DATE,AS_OF_DATE,PERIOD_END_DATE,ORIG_IDS,COUNT(GROUP(BS_OPTIONS_GRANTED()))
ID,,,,,
IdentityGroup,2020-04-30,2020-05-01,2020-04-04,None,171


In [56]:
params = {'start': '2015-12-31','end':'2016-01-31', 'frq':'d'} 
 
eqy_dvd_yld_12m = bq.data.dividend_yield(**params) 
 
avg_eqy_dvd_yld = eqy_dvd_yld_12m.avg() 
gics_sector = bq.data.gics_sector_name() 
 
group_gics_dvd_yld = bq.func.group(eqy_dvd_yld_12m, 
                                   by=gics_sector).avg() 
 
ungroup_gics_dvd_yld = bq.func.ungroup(group_gics_dvd_yld) 
 
fields = [avg_eqy_dvd_yld, ungroup_gics_dvd_yld] 
 
universe = bq.univ.members('INDU Index') 
request = bql.Request(universe, fields)

In [57]:
response = bq.execute(request)
df1 = bql.combined_df(response)
df1

,DATE,"AVG(DIVIDEND_YIELD(start=2015-12-31,end=2016-01-31,frq=PER.d))",GICS_SECTOR_NAME(),ORIG_IDS,"UNGROUP(AVG(GROUP(DIVIDEND_YIELD(start=2015-12-31,end=2016-01-31,frq=PER.d),by=GICS_SECTOR_NAME())))"
ID,,,,,
AXP UN Equity,2016-01-31,0.018470,None,None,NaN
VZ UN Equity,2016-01-31,0.048325,None,None,NaN
BA UN Equity,2016-01-31,0.028356,None,None,NaN
CAT UN Equity,2016-01-31,0.048121,None,None,NaN
JPM UN Equity,2016-01-31,0.029265,None,None,NaN
CVX UN Equity,2016-01-31,0.051054,None,None,NaN
KO UN Equity,2016-01-31,0.031400,None,None,NaN
DIS UN Equity,2016-01-31,0.014043,None,None,NaN
XOM UN Equity,2016-01-31,0.037807,None,None,NaN


In [64]:
bq.univ?

Type:        property
String form: <property object at 0x0000025168E66C28>
Docstring:  
An accessor property for all BQL universe handlers.

You can use this method to access BQL universe handlers
to use in requests.

Example
-------
>>> bq = bql.Service()
>>> index = bq.univ.members("INDU Index")
>>> price = bq.data.px_last()
>>> request = bql.Request(index, price)
>>> response = bq.execute(request)
BQL response object

|


Find the 30 day moving average for all valid trading days (excludes weekends). 

In [67]:
params = {'start': '2016-01-01',                    
          'end': '2016-12-31',                    
          'fill': 'na',                    
          'frq': 'd'} 
 
px_last = bq.data.px_last() 
drop_na = bq.func.dropna(px_last) 
window = bq.func.window(drop_na, windowsize=30) 
avg = bq.func.avg(window) 
 
ticker = 'FB US Equity' 
 
request = bql.Request(ticker, avg, with_params=params)
response = bq.execute(request)

In [68]:

df1 = bql.combined_df(response)
df1

,DATE,CURRENCY,ORIG_IDS,winDuration,winSize,WINDOW_ID,"AVG(WINDOW(DROPNA(PX_LAST()),windowsize=30))"
ID,,,,,,,
FB US Equity:W0,2016-02-16,USD,FB US Equity,44,30,W0,101.447667
FB US Equity:W1,2016-02-17,USD,FB US Equity,44,30,W1,101.547000
FB US Equity:W2,2016-02-18,USD,FB US Equity,44,30,W2,101.571667
FB US Equity:W3,2016-02-19,USD,FB US Equity,44,30,W3,101.625000
FB US Equity:W4,2016-02-22,USD,FB US Equity,46,30,W4,101.933000
FB US Equity:W5,2016-02-23,USD,FB US Equity,44,30,W5,102.204000
FB US Equity:W6,2016-02-24,USD,FB US Equity,44,30,W6,102.516333
FB US Equity:W7,2016-02-25,USD,FB US Equity,44,30,W7,102.806333
FB US Equity:W8,2016-02-26,USD,FB US Equity,44,30,W8,103.222333


In [71]:
bql.Request?

Init signature: bql.Request(universe, items, with_params=None, preferences=None)
Docstring:     
Create a BQL request object.

Parameters
----------
universe
    A BQL item or list of items that describes a security universe, an
    iterable of security ticker strings, or a single security string.  The
    universe values are included in the `FOR` clause of the BQL query
    string.
items
    A BQL data item or list of BQL data items.  The items values are
    include in the `GET` clause of the BQL query string.
with_params
    A dict that maps parameter names to values. These parameter values are
    put in the `WITH` clause of the BQL query string.
preferences
    A mapping of preference parameter names to values.

Returns
-------
BQL request object


Example
-------
>>> bq = bql.Service()
>>> index = bq.univ.members("INDU Index")
>>> price = bq.data.px_last()
>>> request = bql.Request(index, price)
>>> response = bq.execute(request)
BQL response object

|
File:           c:\blp\bqua